In [1]:
# other imports
import sys
sys.path.append('..')
from os.path import join, exists
import numpy as np
from PIL import Image
import glob
import click

# PyTorch imports
import torch
from torch import cuda
from torch.autograd import Variable

# own script imports
import training.ds_transformations as td
import training.helpers as hp

In [2]:
class Config():
    """
    Configuration Class in which all necessary parameters that will be used in the further process are defined.
    """
    DEVICE = hp.get_device()
    DATASET_DIR = '../../ear_dataset'
    MODEL_DIR = '../../models/ve_g_margin_2,0.pt'
    is_small_resize = False
    DATABASE_FOLDER = '../../embeddings/'

In [3]:
# At this point, you select the person ( the one from whom the new images were taken) 
# from whom embeddings are to be created in the following.
# The entrclicky is made via the index in front of the name.
new_person = hp.choose_folder(dataset_path=Config.DATASET_DIR)
check = exists(join(Config.DATABASE_FOLDER, new_person+'.npy'))

Index   Name                
0       konrad_von          
1       falco_len           


In [4]:
# Now it is checked whether embeddings are already stored by the person 
# and the user is informed if necessary.
# The user then decides, by y/n confirmation, how to proceed.

if check: print('The embedding of ' + new_person + ' already exists. Please check the folder first!')
if click.confirm('Do you want to continue?', default=True): 
    pass
else: 
    sys.exit(0)

The embedding of falco_len might already exist. Please check the folder first!
Do you want to continue? [Y/n]: 

SystemExit: 0

In [5]:
# Load the model that will be used to create the embeddings.
model = torch.load(Config.MODEL_DIR, map_location=torch.device(Config.DEVICE))
# Specify a set of transformations to be applied to all captured images before creating embeddings.
transformation = td.get_transform('siamese_valid_and_test', Config.is_small_resize)

/Users/falcolentzsch/.virtualenvs/Bachelorthesis/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torchvision.models.mobilenet.MobileNetV2' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/falcolentzsch/.virtualenvs/Bachelorthesis/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/falcolentzsch/.virtualenvs/Bachelorthesis/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modu

In [6]:
def pipeline(input_, preprocess):
    """
    This method performs a series of image processing procedures. It also checks whether one of the tensor in the
    following can be processed on the graphics card.
    1. convert the input to gray image
    2. perform preprocessing (in this case defined in the transformations
    3. sizes adjustment
    4. rearrange the tensor
    """
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.is_small_resize)[0], td.get_resize(Config.is_small_resize)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)   
    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [7]:
# Here, each image is now converted into an embedding. 
# First, the images are preprocessed, then processed through the network and converted into an embedding
# Finally the Embeddings are saved in our embeddings database.
embeddings = []
image_list = []
for filename in glob.glob( join(Config.DATASET_DIR, new_person, '*') ):
    img = Image.open(filename)
    img_processed = pipeline(img,transformation)
    image_list.append(img_processed)
    
embeddings = np.array([model(Variable(i)).cpu() for i in image_list])
    
np.save( join(Config.DATABASE_FOLDER,new_person+'.npy'), embeddings)  

/Users/falcolentzsch/.virtualenvs/Bachelorthesis/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
